# Tabular Data Analysis 2: Basic operations in pandas

The previous notebook gave an overview of how the pandas library enables to load, manipulate and visualize a dataset.

This notebook introduces the basic operations for data manipulation in pandas, before we focus on data visualization in the next notebook.

## Preliminary: Access and load the data

We need to execute the same code as in the previous notebook to load the pandas library and the Open Food Facts dataset.

In [ ]:
# (just execute this cell)

# enable Colab to access files (here shortcuts) on your Drive
# from google.colab import drive

# drive.mount("/content/drive")

In [ ]:
# (just execute this cell)

# import pandas
import csv
import pandas as pd

# we need this data type for ordered categoricals
from pandas.api.types import CategoricalDtype

# lift some limitations in column width, so more cell values are displayed in full
pd.set_option("display.max_colwidth", 110)

# dataset and data type of the columns
FOLDER = "../data/processed"  # "drive/MyDrive"
OFF_FILE = f"{FOLDER}/off_products_subset.csv"
DTYPE_FILE = f"{FOLDER}/dtype.txt"


# custom function to load the Open Food Facts subset
def load_off():
    """Load the filtered subset of Open Food Facts.

    Returns
    -------
    df : pd.DataFrame
      (A filtered subset of the) Open Food Facts tabular dataset.
    """
    # load the data types for the columns
    with open(DTYPE_FILE) as f:
        dtype = eval(f.read())

    # load the dataset
    df = pd.read_csv(OFF_FILE, sep="\t", dtype=dtype, quoting=csv.QUOTE_NONE)
    # convert columns with datetimes
    for col_name in (
        "created_datetime",
        "last_modified_datetime",
        "last_updated_datetime",
        "last_image_datetime",
    ):
        # ISO 8601 dates
        df[col_name] = pd.to_datetime(df[col_name])
    #
    return df


# load the dataset (takes around 50 seconds)
df = load_off()

### Making a selection into a proper DataFrame

You can manipulate each of these selections as a DataFrame, but behind the scenes, they are *views* of the original DataFrame `df`.
The *view* mechanism avoids unnecessary copies of the dataset, but it is problematic when we really want to extract a subset and perform some operations only on this subset.

For instance, let us select all products in `df` with sugars and fat per 100g greater than 0, and add a column with the sugars to fat ratio.

First, we need to define two filtering conditions and apply them jointly using the [boolean "and" `&`](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#boolean-indexing). Store the result in a variable named `df_sugarsfat`.

Now, let us try to add to `df_sugarsfat` a new column named `"sugarsfat_ratio"` with the sugars to fat ratio.

The output seems fine, but it is preceded by a `SettingWithCopyWarning` that tells us we are working on a *view* when we should be working on an independent copy of the subset of the dataframe.

To avoid this warning, we need to turn our selection into an independent dataframe, with the function `copy()`, and store the result in our variable named `df_sugarsfat`:

Then let us add to (our new) `df_sugarsfat` a column named `"sugarsfat_ratio"` with the sugars to fat ratio.

The ratios are the same as before, except we got rid of the big warning, so we must be doing things the *right* way.

We will not go further and we certainly do not expect you to master the [difference between a view and a copy](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy), but at least now you know that if you encounter the big scary warning, you probably need to `copy()` your selection of rows.

### Selecting subsets

One of the fundamental operations on DataFrames is to be able to filter the dataset on a certain condition, to keep only certain rows or columns.

The basic operators for selection are square brackets `[]`, `loc` and `iloc`, and you can select rows or columns by their position or label, or with a conditional expression on values, see the [pandas intro tutorial 03](https://pandas.pydata.org/docs/getting_started/intro_tutorials/03_subset_data.html).

Filter rows in `df` to keep only products with Nutri-Score 'a', and store the result in a variable called `df_nutri_a`.


In [ ]:
# (just execute this cell)
df_nutri_a = df[df["nutriscore_grade"] == "a"]
df_nutri_a

You should have 56,260 entries.

Now filter rows in `df` to keep products whose quantity of sugars per 100g is higher than 20g, and store the result in a variable called `df_sugar_gt20`.

You should obtain 83891 entries.

Filter the dataset `df` to keep only the columns corresponding to the :
* barcode,
* url,
* date of creation,
* product name,
* brands,
* categories,
* ingredients text,
* main category,
* Nutri-Score grade,
* Nutri-Score score,
* Nova group.

And store the result in a variable named `df_sel_cols`.

### Renaming columns

Column names are not always ideal, either because they are not transparent (it is hard for you or an external user to understand what they stand for) or because they would look bad if they were used directly to label the axes of a datavisualization.

pandas provides means to rename columns, see the [pandas intro tutorial 05](https://pandas.pydata.org/docs/getting_started/intro_tutorials/05_add_columns.html).

### Summary statistics

You can compute various summary statistics that depend on the type of variable in each column, see the [pandas intro tutorial 06](https://pandas.pydata.org/docs/getting_started/intro_tutorials/06_calculate_statistics.html).

Compute summary statistics for several columns from different types, and combinations of columns that could provide interesting insights.

For instance, compute the means of the nutritional values in `df` for :
* fat,
* saturated fat,
* sugars,
* salt.

### Computing on columns

You can manipulate columns in various ways, including with operations that apply element-wise as we saw for NumPy arrays in the first notebook.

You can for instance subtract the mean value of a column to each value in the column.

### Sorting data

The entries are sorted by barcode.
We might find it easier to understand the dataset if we sort entries by another criterion.

### Working with dates

pandas has a specific data type for dates. You can explicitly ask pandas to use this type for specific columns, either during `read_csv` or after (as I did in `load_off`), see the [pandas intro tutorial 09](https://pandas.pydata.org/docs/getting_started/intro_tutorials/09_timeseries.html).

This specific data type makes it easy to filter entries by the month of their creation, to know what day of the week an entry was created, or to sort entries by their date of creation.

Sort entries by their date of creation, and store the result in a variable named `df_sort_created`.

### Working with textual data

pandas provides a number of functions to process text strings, see the [pandas intro tutorial 10](https://pandas.pydata.org/docs/getting_started/intro_tutorials/10_text_data.html).

Use these functions to select all entries whose list of brands contains "Casino" (this operation is case-sensitive, so mind the initial capital letter!), and store the result in a variable named `df_casino`.

### Wrapping it all together

Select all the products that are in the category for spreads and store this subset in a variable `df_spreads`.

## Bonus exercise : Traffic light labelling

The [traffic light labelling system](https://www.nutrition.org.uk/healthyliving/helpingyoueatwell/324-labels.html?start=3) is used on the [Open Food Facts website (French)](https://fr.openfoodfacts.org/reperes-nutritionnels) to display colorful, easier to grasp information on 4 nutritional values with a color code :

* fat,
* saturated fat,
* sugars,
* salt.

## To go further

### Python for data science

* [Programming in Python for Data Science](https://prog-learn.mds.ubc.ca/en/)
* [Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/)

The OpenFoodFacts dataset does not contain these indicators, but you can recompute them from the [reference table](https://www.nutrition.org.uk/media/er5n0c3s/capture.png).

Add 4 columns to the dataset, one for each of the 4 relevant nutritional values, that will contain the  (low, medium, high) or color (green, yellow, red) of the traffic light.

> **HINT** You can simplify the exercise and express all conditions on the values per 100g (ignoring the rightmost column of the table where thresholds are expressed per portion).

We can use [`loc`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.loc.html?highlight=loc#pandas.DataFrame.loc), see "Setting values".

In [ ]:
# (just execute this cell)
# for fat_100g
df["tl_fat"] = "unknown"
df.loc[df["fat_100g"] <= 3, "tl_fat"] = "green"
df.loc[(df["fat_100g"] > 3) & (df["fat_100g"] <= 17.5), "tl_fat"] = "amber"
df.loc[(df["fat_100g"] > 17.5), "tl_fat"] = "red"

Let us check that the traffic lights for fat are as wanted.

In [ ]:
# (just execute this cell)
df[["fat_100g", "tl_fat"]].head(10)

Now you can define the traffic lights for the 3 remaining nutritional values, in columns `"tl_saturated-fat"`, `"tl_sugars"`, `"tl_salt"`.

We can display the traffic lights for the first 10 products, and compare with what the Open Food Facts website displays (remember: you can retrieve URLs from the column `url`).

> **HINT** If you can't find the right pattern to look for, take a peak at the spelling of the categories: Print the content of the column and browse through the values until you find a suitable value.

You should find 25565 spreads.

For these spreads, compute the means of the nutritional values for :
* fat,
* saturated fat,
* sugars,
* salt.

You should find the mean values :

* fat = 20.96 g,
* saturated-fat = 7.97 g,
* sugars = 29g,
* salt = 0.77g.


For each of these 4 nutritional values, compute the percentage of difference between each product and the average of its category, and store the computed values as new columns to `df_spreads` (eg. `diff-fat_100g`, `diff-sugars_100g` etc).

Remember that you can find help in the [pandas intro tutorial 05](https://pandas.pydata.org/docs/getting_started/intro_tutorials/05_add_columns.html) and [pandas tutorial 06](https://pandas.pydata.org/pandas-docs/stable/getting_started/intro_tutorials/06_calculate_statistics.html#min-tut-06-stats).

Note that these values differ from what the Open Food Facts website displays when you look at the nutritional values of a product from this category, eg. [Coconut Spread - premium Srikaya - Hey Boo - 227 g](https://world.openfoodfacts.org/product/0608938316165/coconut-spread-premium-srikaya-hey-boo).

In [ ]:
# (uncomment this line and check the output)
# df_spreads[df_spreads['code'] == '0608938316165']['diff-fat_100g']

This product contains barely 0.2% more fat than the average spreads in our subset `df_spreads`, but 17% more than the average spreads in the entire Open Food Facts dataset (as displayed on the OFF website).


This is because the Open Food Facts website uses its entire dataset, whereas we are working on a filtered subset of "reasonably complete" product entries prepared beforehand to keep only products with :

* a non-ambiguous barcode in the EAN-8 or EAN-13 formats ;
* a product name,
* brands,
* an image URL for the product ;
* a category ;
* basic nutritional values.

It seems that, in this "resonably complete" subset, spreads contain more fat on average than in the whole Open Food Facts dataset.

Is the entire Open Food Facts dataset closer to the reality of what is on the shelves of supermarkets ?
Is our subset more faithful globally ? Is it more faithful to the consumer market in certain countries, eg. France and Spain ?

These questions raise the more general problem of [Selection bias](https://en.wikipedia.org/wiki/Selection_bias) that lies behind every data analysis and use of dataset for eg. artificial intelligence systems.

You should get 4434 products whose brands contains "Casino".

Let us check that the first and last entries are as expected.

Display the first entries.

The oldest entries in our dataset date from 2012.

Display the last entries.

The newest entries in our dataset date from 2021-08-15 (when I downloaded the entire dataset).

Sort entries by brand, following the [pandas intro tutorial 07](https://pandas.pydata.org/docs/getting_started/intro_tutorials/07_reshape_table_layout.html), and store the result in a variable named `df_sort_brands`.

Let us look at the brands for the first entries, sorted by brands.

Oddly, only the first few lines have brand names that start with a letter, then brand names start with a special character (`!` or `#`).
This is unexpected, because special characters should appear first.

What happened here ? Let us have a better look at the *values* in the `brands` column of our sorted dataframe `df_sort_brands`.

In [ ]:
# (just execute this cell)
df_sort_brands["brands"].head(20).values

In the first entries, the `brands` value starts with a whitespace.
This explains why they were sorted before the entries whose `brands` start with a special character.

Brand names rarely (if ever) start with a whitespace, hence we can assume that whoever added these products made a typing error.

⚠ Datasets contain all sorts of errors and oddities. Datasets released by public agencies or big actors are usually cleaner than crowdsourced datasets, but you should always be cautious.

To confirm our hypothesis and check whether the entries are properly sorted, we can use `iloc` to retrieve entries at arbitary positions in the DataFrame.

For instance, let us check the entries ranked 4881 to 4899 (or 4900 excluded).

In [ ]:
# (just execute this cell)
df_sort_brands["brands"].iloc[4881:4900]

The sorted brands are `Alba`, `Alba torri e sapori`, `Albacore` then `Albalact`, which is what we were expecting.

Sort entries by the Nutri-Score grade, and store the result in a variable named `df_sort_nsgrade`.

Let us check the first 20 entries.

The entries with nutriscore grade 'a' are ranked first, as expected.

Sort entries by the Nutri-Score grade and Nova group (together), and store the result in a variable named `df_sort_nsgrade_novagroup`.

Let us check the first 20 entries.

Products with nutriscore_grade 'a' and nova_group '1' appear first.

Let us rename each of the columns whose name ends with `_en`.

First, we need to list such columns.

In [ ]:
# (just execute this cell)
# list the column names that end with _en
cols_en = [x for x in df.columns if x.endswith("_en")]
cols_en

['categories_en',
 'origins_en',
 'labels_en',
 'countries_en',
 'traces_en',
 'additives_en',
 'states_en',
 'main_category_en']

Now we can `rename` each of the columns ending with `_en`, so as to drop this suffix.

For instance, `main_category_en` should be renamed `main_category`.

We can store the result in a variable named `df_ren_en`.

To see if it worked, let us display the column names in `df_ren_en` and check that our `_en` columns, such as `main_category_en`, have been renamed as expected.